# 二項分布モデルでのP値

* 黒木玄
* 2025-11-26

<h1>目次<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#二項分布でのP値に関する解説の例" data-toc-modified-id="二項分布でのP値に関する解説の例-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>二項分布でのP値に関する解説の例</a></span><ul class="toc-item"><li><span><a href="#5⃣-STEP-4:-P値の定義" data-toc-modified-id="5⃣-STEP-4:-P値の定義-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>5⃣ STEP 4: P値の定義</a></span></li><li><span><a href="#P値の計算の仕方" data-toc-modified-id="P値の計算の仕方-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>P値の計算の仕方</a></span></li><li><span><a href="#両側P値の計算練習問題" data-toc-modified-id="両側P値の計算練習問題-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>両側P値の計算練習問題</a></span></li></ul></li></ul></div>

In [37]:
ENV["LINES"] = 200
ENV["COLUMNS"] = 200
using Distributions
using StatsPlots
default(fmt=:png)

x ⪅ y = x < y || x ≈ y
x ⪆ y = x > y || x ≈ y

"""
    central法による二項分布の両側P値
二項分布binにおける「k以下になる確率の2倍」と「k以上になる確率の2倍」と「1」の中での最小の値

このP値からClopper-Pearsonの信頼区間が得られる。
"""
pvalue_central(bin, k) = min(2cdf(bin, k), 2ccdf(bin, k-1))

"""
    minimum likelihood法による二項分布の両側P値
このP値からSterneの信頼区間が得られる。
"""
function pvalue_minlike(bin, k)
    supp = support(bin)
    sum(pdf(bin, i) for i in supp if pdf(bin, i) ⪅ pdf(bin, k))
end

"""
    score法による二項分布の両側P値
このP値は二項分布の正規分布近似で定義された両側P値と同じ。

このP値はWilsonのscore信頼区間を与える。
"""
function pvalue_score(bin, k)
    (; n, p) = bin
    phat = k/n
    se = sqrt(p * (1 - p) / n)
    z = (phat - p) / se
    2ccdf(Normal(), abs(z))
end

"""
    Wald法による二項分布の両側P値
このP値も二項分布の正規分布近似で定義された両側P値だが、データ \$n, k\$ から推定する必要がない \$\\hat{p}=k/n\$ の標準偏差を推定してしまっているので、score法の両側P値よりも精度が低い。

このP値は教科書によく書いてある母比率に関するWaldの信頼区間を与える。
"""
function pvalue_wald(bin, k)
    (; n, p) = bin
    phat = k/n
    sehat = sqrt(phat * (1 - phat) / n)
    z = (phat - p) / sehat
    2ccdf(Normal(), abs(z))
end

bin = Binomial(20, 0.6)
ks = (7, 8, 16, 17)
@show pvalue_central.(bin, ks)
@show pvalue_minlike.(bin, ks)
@show pvalue_score.(bin, ks)
@show pvalue_wald.(bin, ks)
;

pvalue_central.(bin, ks) = (0.04205785495554228, 0.1130527340685061, 0.10190390638833315, 0.031922325580016536)
pvalue_minlike.(bin, ks) = (0.03699009026777941, 0.10747832022841937, 0.07198088067193767, 0.022427038142043273)
pvalue_score.(bin, ks) = (0.02247887336612527, 0.06788915486182907, 0.06788915486182893, 0.02247887336612527)
pvalue_wald.(bin, ks) = (0.0190763221017784, 0.06788915486182907, 0.025347318677468197, 0.0017414012528258463)


In [35]:
ccdf(Binomial(20, 0.6), 18-1), 2ccdf(Binomial(20, 0.6), 18-1)

(0.003611472059128875, 0.00722294411825775)

In [36]:
cdf(Binomial(20, 0.6), 6), 2cdf(Binomial(20, 0.6), 6)

(0.00646587535203501, 0.01293175070407002)

## 二項分布でのP値に関する解説の例

以下の例は中原治著『基礎から学ぶ統計学』の「1-5 二項検定」の節の「5⃣ STEP 4: 棄却域と有意水準」以降の説明をP値に関する説明で置き換えることを意識して書かれたものです。

その本の例題1.1では、B薬は60%の患者に効果があることが分かっていると仮定されており、A薬については無作為に選んだ患者20人中18人に効果があったというデータの値が得られていると仮定されています。以下の説明でもこの設定を使います。

### 5⃣ STEP 4: P値の定義

例題1.1の状況でのP値はその値の __小ささ__ が次のように解釈できるように構成されます:

>A薬が無作為に選んだ患者20人中18人に効いたというデータの値が得られたとき、A薬が効く患者の割合がB薬と同じ60%だと信じている人は確率的にどれだけ意外だと感じるか。

そのためには __片側P値__ を、

>効果がある患者の割合が60%だと仮定したとき、無作為に選んだ患者20人中18人 __以上__ に効果がある確率

と定義すると良いです。この確率の値(片側P値の値)は0.36%とかなり小さくなります(計算の仕方は後で説明します)。このことは、A薬とB薬が効く患者の割合が等しいと信じている人にとって、A薬が無作為に選んだ患者20人中18人に効いたというデータの値はかなり意外だと感じられることを意味しています。

__両側P値__ は片側P値の2倍で定義されます。この場合の両側P値は0.72%でやはりかなり小さい。

より正確には、この両側P値を

>「A薬が効く患者の割合が60%である」という検定仮説の「A薬が無作為に選んだ患者20人中18人に効いた」というデータの値に関する両側P値

と呼びます。この両側P値の小ささは

>A薬が効く患者の割合が60%であると信じている人が、A薬が無作為に選んだ患者20人中18人に効いたというデータの値に感じる意外さ

を表しています。

以下では単にP値と言ったら、両側P値を意味するものとします。

P値は小さいほど、仮説に対するデータの値の意外さが大きいことを意味します。

__片側確率の2倍の両側P値の正確な定義と欠点:__ 「無作為に選んだ $n$ 人中 $k$ 人に薬が効いた」というデータの値が得られたときの、片側確率の2倍で定義された「薬が効く患者の割合は$p$である」という仮説の両側P値の正確な定義は

>薬が効く確率が $p$ であるという仮定の下で、無作為に選んだ $n$ 人中 $k$ 人以上に効く確率の2倍と無作為に選んだ $n$ 人中 $k$ 人以下に効く確率の小さい方の値

です。このように定義された両側P値の欠点は

* 無用に値が大きくなり過ぎるという性質を持つ。
* $1$ より大きくなることがある。

P値は確率だとみなしたい値なので0以上1以下になって欲しいのですが、上の定義だと1より大きくなってしまう場合が出て来てしまいます。その欠点をなくすために、

>薬が効く確率が $p$ であるという仮定の下で、無作為に選んだ $n$ 人中 $k$ 人以上に効く確率の2倍と無作為に選んだ $n$ 人中 $k$ 人以下に効く確率と __1__ の中での最小値

と定義することも多いです。

__片側P値だけではなく、両側P値を考えたくなる理由:__ A薬が無作為に選んだ患者20人中60%の12人よりもずっと大きな人数(例えば18人)に効果があった場合だけではなく、その反対側のずっと小さな人数(例えば5人)にしか効果がない場合にも、A薬が効く患者の割合が60%だと信じている人にとっては意外だと感じられるでしょう。このことから、無作為に選んだ患者20人中60%の12人からデータの値以上に離れる確率の小ささで、「A薬が効く患者の割合は60%」という仮説を信じている人から見たデータの値の意外さを評価するという考え方も自然だということに気付きます。そのような考え方で定義されるP値が両側P値です。上ではシンプルに反対側の確率はもとの片側確率に等しいだろうという大雑把な想定のもとで、片側確率の2倍で両側P値を定義しました。モデルの確率分布が左右非対称な場合には他にも無数の選択肢があるのですが、そういう問題については統計学の技術的側面についてもっと詳しくなってから考えれば十分です。

__尤度とP値の違いに注意:__ 上では片側P値を「効果がある患者の割合が60%だと仮定したとき、無作為に選んだ患者20人中18人 __以上__ に効果がある確率」と定義し、両側P値をその2倍で定義しました。読者の中には太字になっている「以上」を削除して、「効果がある患者の割合が60%だと仮定したとき、無作為に選んだ患者20人中18人に効果がある確率」を考えたくなる人がいると思います。その人は鋭い！ 「以上」の語を取り去ってできる確率は __尤度(ゆうど)__ と呼ばれており、統計学的に非常に基本的かつ重要な量になっています。尤度とP値を混同しないように注意して下さい。

### P値の計算の仕方

__二項係数の記号法:__ 以下では二項係数を

$$
\binom{n}{k} = {}_nC_k = \frac{n!}{k!(n-k)!}
$$

と表すことにします。例えば, $0!=1$ に注意すると、

$$
\begin{aligned}
&
\binom{20}{20} = \frac{20!}{20!0!} = \frac{20!}{20!} = 1,
\\ &
\binom{20}{19} = \frac{20!}{19!1!}
=\frac{20\cdot 19\cdot 18\cdots 2\cdot 1}{19\cdot 18\cdots 2\cdot 1\times 1}
= 20,
\\ &
\binom{20}{18} = \frac{20!}{18!2!}
=\frac{20\cdot 19\cdot 18\cdot 17\cdots 2\cdot 1}{18\cdot 17\cdots 2\cdot 1\times 2}
= 190.
\end{aligned}
$$

${}_nC_k$ は高校数学では標準的な記号ですが、専門的な文献では $\binom{n}{k}$ の方がよく使われているように思われるので主に $\binom{n}{k}$ を使います。どちらを使うかは好みの問題でしかないので、$\binom{n}{k}$ という書き方が嫌いな人は ${}_nC_k$ と書いても構いません。

片側P値の

>効果がある患者の割合が60%だと仮定したとき、無作為に選んだ患者20人中18人 __以上__ に効果がある確率

は次のように計算される。

* (20人中$k$人に効果がある確率) $= \binom{20}{k}0.6^{k}(1-0.6)^{20-k}$

なので、関数電卓その他の方法で計算すると、

* (20人中20人に効果がある確率) $= \binom{20}{20}0.6^{20}(1-0.6)^0 = 0.6^{20} \approx 3.6561584400629733\mathrm{e-}5$
* (20人中19人に効果がある確率) $= \binom{20}{19}0.6^{20}(1-0.6)^1 = 20\cdot 0.6^{19}\cdot 0.4 \approx 0.00048748779200839646$
* (20人中18人に効果がある確率) $= \binom{20}{18}0.6^{20}(1-0.6)^2 = 190\cdot 0.6^{18}\cdot 0.4^2 \approx 0.00048748779200839646$

ここで $3.6561584400629733\mathrm{e-}5$ は $3.6561584400629733\times 10^5$ を意味する。

上の3つの値の和を計算ことによって次を得る:

* (20人中18人以上に効果がある確率) $\approx 0.003611472059128871$

これが片側P値である。この2倍の両側P値は

* (両側P値) $= 0.007222944118257742$

以上によって片側P値は約0.36%で両側P値が約0.72%になることが分かった.

In [33]:
@show p20 = binomial(20, 20) * 0.6^20 * 0.4^0
@show p19 = binomial(20, 19) * 0.6^19 * 0.4^1
@show p18 = binomial(20, 18) * 0.6^18 * 0.4^2
@show pval_one_sided = p20 + p19 + p18
@show pval_both_sided = 2pval_one_sided;

p20 = binomial(20, 20) * 0.6 ^ 20 * 0.4 ^ 0 = 3.6561584400629733e-5
p19 = binomial(20, 19) * 0.6 ^ 19 * 0.4 ^ 1 = 0.00048748779200839646
p18 = binomial(20, 18) * 0.6 ^ 18 * 0.4 ^ 2 = 0.003087422682719845
pval_one_sided = p20 + p19 + p18 = 0.003611472059128871
pval_both_sided = 2pval_one_sided = 0.007222944118257742


### 両側P値の計算練習問題

__問題:__ 「薬Aが効く患者の割合は60%である」という仮定の下で「薬Aは無作為抽出された20人中$k$人に効いた」というデータの値が得られる確率$P(k)$を0以上20以下のすべての整数$k$について求め、確率の数表を作成せよ。

__解答例:__ 二項分布の確率 $P(k) = \binom{20}{k} 0.6^k 0.4^{20-k}$ を $k=0,1,2,\ldots,20$ について求めればよい. その結果は次のようになる:

|k|P(k)||k|P(k)|
|---:|:---|---|---:|:---|
|0 | 1.09951e-8|　|12 | 0.179706|
|1 | 3.29853e-7||13 | 0.165882|
|2 | 4.70041e-6||14 | 0.124412|
|3 | 0.0000423037||15 | 0.074647|
|4 | 0.000269686||16 | 0.0349908|
|5 | 0.00129449||17 | 0.0123497|
|6 | 0.00485435||18 | 0.00308742|
|7 | 0.0145631||19 | 0.000487488|
|8 | 0.0354974||20 | 0.0000365616|
|9 | 0.0709949||||
|10 | 0.117142||||
|11 | 0.159738||||

ただし、1.09951e-8は1.09951×10⁻⁸を意味する。以上の結果はWolframAlphaに

```mathematica
Table[k, {k, 0, 20}], Table[Binomial[20, k]*0.6^k*0.4^(20-k), {k, 0, 20}] *2
```

と入力することによって得られる([WolframAlphaで実行](https://www.wolframalpha.com/input?i=Table%5Bk%2C+%7Bk%2C+0%2C+20%7D%5D%2C+Table%5BBinomial%5B20%2C+k%5D*0.6%5Ek*0.4%5E%2820-k%29%2C+%7Bk%2C+0%2C+20%7D%5D+&lang=ja)).

<img width=70% src="https://raw.githubusercontent.com/genkuroki/public/refs/heads/main/0055/WolframAlpha%EF%BC%BFBinomial_20_0.6.png">

In [51]:
Any[0:11 pdf.(Binomial(20, 0.6), 0:11) 12:23 pdf.(Binomial(20, 0.6), 12:23)]

12×4 Matrix{Any}:
  0  1.09951e-8   12  0.179706
  1  3.29853e-7   13  0.165882
  2  4.70041e-6   14  0.124412
  3  4.23037e-5   15  0.074647
  4  0.000269686  16  0.0349908
  5  0.00129449   17  0.0123497
  6  0.00485435   18  0.00308742
  7  0.0145631    19  0.000487488
  8  0.0354974    20  3.65616e-5
  9  0.0709949    21  0.0
 10  0.117142     22  0.0
 11  0.159738     23  0.0

__問題:__ 「薬Aは無作為に抽出された患者20人中6人に効いた」というデータの値に関する「薬Aが効く患者の割合は60%である」という仮説の両側P値を求めよ。ただし、その両側P値は片側確率の2倍で定義されたものとする。その計算で上の問題で作成した確率の数表を用いて良い。

__解答例:__ 求めるべき両側P値は、「薬Aが効く患者の割合は60%である」という仮定の下で「薬Aが効いたの人数は無作為抽出された20人中6人以下であった」となる確率の2倍である。ゆえに、求めたい両側P値は、上の問題で求めた確率$P(k)$を$k=0,1,2,3,4,5,6$について足し合わせて2倍すれば求まる。

In [47]:
(1.09951e-8 +
3.29853e-7 + 
4.70041e-6 + 
0.0000423037 + 
0.000269686 + 
0.00129449 + 
0.00485435) * 2

0.012931741916200001

すなわち、求めたい両側P値は約1.3%である。

In [48]:
pvalue_central(Binomial(20, 0.6), 6)

0.01293175070407002